<h1 align="center">Espécies Ameaçadas de Extinção no Brasil</h1>

<h3>1. Introdução</h3>
<p style="text-align:justify">A Portaria MMA 300/2022 revogou as Portarias 443/2014, 444/2014 e 445/2014, as quais em seua anexos definiam as listas de espécie da flora, fauna e peixes e invertebrados aquáticos ameaçados de extinção no Brasil. Estas listas foram atualizadas e hoje fazem parte da Portaria MMA 300/2022. Os dados aqui utilizados foram obtidos em 14 de dezembro de 2022.</p>
<p style="text-align:justify"></p>
<p style="text-align:justify">O objetivo deste <i>notebook</i> é utilizar a técncia <i>web scraping</i> para extrair as tabelas referentes as espécies da flora, fauna brasileiras ameçadas de extinção, conforme <a href="http://www.in.gov.br/web/dou/-/portaria-gm/mma-n-300-de-13-de-dezembro-de-2022-450425464">Portaria MMA nº 300</a>, de 13 de dezembro de 2022. Para tal utilizaremos a linguagem de programação R</p>

### 2. Import pacotes necessários

In [ ]:
# Import libraries
library(rvest)
library(dplyr, warn.conflicts = FALSE)
library(ggplot2)
library(stringr)
library(finch)

### 3. Definição do endereço web

In [ ]:
# Set url
url = 'https://www.in.gov.br/en/web/dou/-/portaria-gm/mma-n-300-de-13-de-dezembro-de-2022-450425464'

### 4. Aplicando a técnica _web scraping_

In [ ]:
# web scraping the url
tab_lst <- url %>%
        read_html() %>%
        html_nodes(xpath = '//table') %>%
        html_table(header = FALSE)

### 5. Obter informações dos dados

In [ ]:
glimpse(tab_lst)

In [ ]:
length(tab_lst)

In [ ]:
# FLORA - lists 1-65
head(tab_lst[[65]])

In [ ]:
# Fauna - lists 66-95
tail(tab_lst[[83]])

<p style="text-align=justify">Ao inspercionar os dados podemos verificar que temos uma lista composta por 96 <i>dataframes</i>, sendo que o último é um dado que não faz parte das tabelas da fauna e flora, o que deveremos excluí-lo da lista.</p>
<p style="text-align=justify">Também é possível verificar que os elementos da lista de 1 a 65 são referentes as tabelas da flora, o restante refere-se as espécies da fauna. Os dataframes de flora possuem 5 colunas, enquato que os da fauna 6</p>

In [ ]:
# Remove undesirable data
tab_lst[[96]] <- NULL

### 5. Limpeza de dados
<p style="text-alig:justify">Precisamos redefinir os nomes das colunas para os dados de fauna e flora, pois possuem número de colunas diferentes, o que permitira unir todos em um único <i>dataframe</i></p>

In [ ]:
# Set column names for each dataframe in a list
for (i in 1:length(tab_lst)) {
    if (length(unlist(attributes(tab_lst[[i]])[2])) == 5) {
        names(tab_lst[[i]]) <- c('n', 'port443', 'familia',
                                 'especie_subespecie_var', 
                                 'categoria')
    }
    else {
        if (length(unlist(attributes(tab_lst[[i]])[2])) == 6) {
            names(tab_lst[[i]]) <- c('n', 'port443', 'ordem',
                                   'familia', 'especie_subespecie', 
                                   'categoria')
        }
    }
}

# Check attributes       
names(tab_lst[[1]])  # flora
names(tab_lst[[94]]) # fauna

### 5.1 Extrair apenas os dados referentes à flora e salvá-los em planilha csv

In [ ]:
# Subset the flora dataframes in the list
df_list <- tab_lst[1:65]

# Loop over the list to merge all flora dataframes in a single dataframe
for (i in 1:65) {
    
    df_list[[i]][1] <- NULL
    df <- bind_rows(df_list)
    df <- df[c(-1, -2), c(1:4)]
    df <- df %>% 
        tibble::rowid_to_column(var = 'n')
    
    df <- df[complete.cases(df), ]
    
    write.csv2(
        df, './port_mma300_2022_flora.csv', 
        row.names = FALSE)
    
}

<p style="text-align:justify">Chama atenção o fato de muitas espécies, tanto da flora quanto da fauna, estarem com a grafia do nome científico errado, a exemplo de <i>Vochysia gummifera</i>, a qual está grafada como <i>Vochysiagummifera</i>, o que pode induzir o público leigo a erros quando busca por expécies ameaçadas. O trecho de código abaixo mostra o número total de espécies da flora que apresentam erro grave de grafia.</p>

In [ ]:
df_flora <- read.csv2('./port_mma300_2022_flora.csv', fileEncoding = 'UTF-8')

sum(is.na(str_extract(df_flora$especie_subespecie_var, 
                               '(\\b\\w+\\s\\w+\\b)|(\\b\\w+\\s\\w+\\s\\w+\\b)')))

### 5.2 Correção dos Nomes Científicos
<p style="text-align:justify">Para corrigir os erros de grafia nos nomes científicos usaremos como referência a lista de espécies do projeto <a href="https://reflora.jbrj.gov.br/reflora">REFLORA, cuja base de dados encontra-se no <a href="http://ipt.jbrj.gov.br/jbrj/archive.do?r=lista_especies_flora_brasil&v=393.291">site do jardim botânico do Rio de Janeiro</a></p>

In [ ]:
require('fuzzyjoin')

# Reflora
reflora <- read.csv2('./reflora20210929.csv',
                     fileEncoding = 'UTF-8')

# Join
tbl_join <- fuzzyjoin::stringdist_inner_join(df_flora, reflora,
                                           by = c('especie_subespecie_var' = 'specie'),
                                           ignore_case = TRUE,
                                           distance_col = 'distance')


df <- tbl_join %>%
        filter(distance < 2) %>%
        distinct(n, .keep_all = TRUE) %>%
        select(c(1:3, 7, 5))


# salvar como csv
write.csv2(df, './port_MMA_300-2022_clean.csv', row.names = FALSE)

## 5.4 Consultar todas as Espécies dos Gêneros _Manilkara_ e _Vochysia_

In [ ]:
# Consultas
df %>%
        filter(specie == str_extract(specie, '(^Manilkara\\s\\w+)*'))

df %>%
        filter(specie == str_extract(specie, '(^Vochysia\\s\\w+)*'))

### 6. Análise dos dados de flora

In [ ]:
# Portaria MMA 443
port443 <- read.csv('D:/data/jbrio/port443.csv')

df_flora <- data.frame(Portaria_443 = nrow(port443),
                       Portaria_300 = nrow(df),
                       Diferença = (nrow(df) - nrow(port443)))

as_tibble(df_flora)

<p style="text-align=justify">Podemos observar que houve um acrescimo de 999 novas espécies ameaçadas de extinção para a flora brasileira.</p>

In [ ]:
# Set figure size
options(repr.plot.width = 12, repr.plot.height = 10)

# Set theme to the plots
my_theme <- theme(plot.title = element_text(hjust = 0.5, size = 18, face = 'bold'),
          axis.title = element_text(size = 17),
          axis.text = element_text(size = 17),
          legend.text = element_text(size = 17),
          legend.position = c(0.9, 0.5))

# Set status dataframe
species_status <- df %>%
        group_by(categoria) %>%
        count(categoria, sort = TRUE)
# Reorder status dataframe by "n" and "categoria" columns
species_status$categoria <- with(species_status, 
                                 reorder(categoria, n, decreasing = TRUE))

# Legend
labels <- c('EN\nEm Perigo', 'VU\nVulnerável', 'CR\nCriticamente em Perigo', 
            'CR (PEX)\nCriticamente em Perigo\n(Provavelmente Extinta)')

status_cod <- c('EN' , 'VU', 'CR' , 'CR (PEX)')

# Status plot        
ggplot(species_status, aes(x = categoria, y = n, fill = n)) +
    geom_col() +
    my_theme +
    scale_x_discrete(limits = status_cod, labels = labels) +
    labs(
            title = 'Espécies da Flora Ameaçadas de Extinção no Brasil',
            x = 'Categoria',
            y = 'Nº de espécies',
            fill = ''
    )

In [ ]:
# Set figure size
options(repr.plot.width = 12, repr.plot.height = 10)

statusFamily <- df %>%
        #filter(familia != "") %>%
        count(familia, sort = TRUE) %>%
        slice_max(n, n = 10)

statusFamily$familia <- with(statusFamily, reorder(familia, n))

ggplot(statusFamily, aes(familia, n)) +
        geom_col(fill = 'steelblue') + coord_flip() +
        geom_text(aes(label = n, hjust = 2), color = 'white', size = 8) +
        my_theme +
        labs(
                title = '10 Famílias da Flora com Maior Número de Espécies Ameaçadas de Extinção',
                x = '',
                y = ''
            )

<p style="text-align:justify">Ao comparar a lista da Portaria MMA 300/2022 com a lista de espécies do projeto REFLORA, foi obervado que há 15 nomes cientificos com grafia de sinônimo taxonômico</p>

## 7. Dados Referentes à Fauna

## 7.1 Extrair apenas os dados referentes à fauna e salvá-los em planilha csv

In [ ]:
# Subset the flora dataframes in the list
df_list_fauna <- tab_lst[66:95]
    
#df_list_fauna <- lapply(df_list_fauna, function(x) { x[!is.na(x)] })
    
df_fauna <- purrr::reduce(df_list_fauna, rbind)
df_fauna <- df_fauna[c(-1, -2), ]     

write.csv2(
    df_fauna, './port_mma300_2022_fauna.csv', 
    row.names = FALSE,
    fileEncoding = 'latin1')

In [ ]:
print(head(df_fauna))
print(tail(df_fauna))

## 7.2 Correção dos Nomes Científicos da Fauna 

<p style="text-align:justify">Asssim como nas tabelas da flora, as da fauna também apresentam erros de grafia nos nomes científico, conforme mostrado na saída do código abaixo.</p>

In [ ]:
df_fauna <- read.csv2('./port_mma300_2022_fauna.csv', fileEncoding = 'latin1')
df_fauna[is.na(str_extract(df_fauna$especie_subespecie, '(\\w+\\s\\w+)')), ]

O código abaixo mostra que foram publicados 235 nomes científicos da fauna de forma incorreta.

In [ ]:
sum(is.na(str_extract(df_fauna$especie_subespecie, '\\w+\\s\\w+')))

Para correção dos nomes científicos da fauna foram utilizados os dados do _[Global Biodiversity Information Facility](https://www.gbif.org/)_.

In [ ]:
# Tabela Fauna GBIF
gbif <- readRDS('./Portaria_MMA_300-2022/gbif_clean.rds')

In [ ]:
port300_fauna <- df_fauna %>%
        mutate(especie_subespecie = tolower(especie_subespecie)) %>%
        mutate(especie_subespecie = R.utils::capitalize(especie_subespecie)) %>%
        mutate(especie_subespecie = str_trim(especie_subespecie)) %>%
        mutate(especie_subespecie = str_squish(especie_subespecie))

In [ ]:
# Nomes incorretos
port300_fauna_erro <- port300_fauna %>%
        filter(!especie_subespecie %in% str_extract(especie_subespecie,
                                                    '(\\b\\w+\\s\\w+\\b)')) %>%
        filter(!especie_subespecie %in% str_extract(especie_subespecie,
                                                    '(\\b\\w+\\s\\w+\\s\\w+\\b)'))

In [ ]:
# Join
tbl_join_fauna <- fuzzyjoin::stringdist_inner_join(port300_fauna_erro, gbif,
                                           by = c('especie_subespecie' = 'especie_GBIF'),
                                           ignore_case = TRUE,
                                           distance_col = 'distance')

# Novo dataframe com os dados processados
df <- port300_fauna %>%
        left_join(tbl_join_fauna, by = 'especie_subespecie')

## Definição das Classes Taxonômicas para as Espécie da Fauna
A obtenção das classes taxômicas foi realizada através dos dados obtidos no site do projeto _The Interim Register of Marine and Nonmarine Genera_ - [IRMNG](https://www.irmng.org/download.php)

In [ ]:
taxon <- readRDS('./taxon_animalia.rds') %>%
    mutate(family = str_trim(family))

glimpse(taxon)

In [ ]:
df_fauna <- read.csv2('./port_MMA_300-2022_fauna_final.csv', fileEncoding = 'latin1')
glimpse(df_fauna)

In [ ]:
# Obter registros duplicados e removê-los
nomes_duplicados <- df_fauna %>%
    count(especie_subespecie, name = 'cnt') %>%
    filter(cnt > 1)

head(nomes_duplicados)
dim(nomes_duplicados)

In [ ]:
# Lista sem nomes duplicados
nrow(filter(df_fauna, !duplicated(especie_subespecie)))

In [ ]:
df_fauna_fim <- taxon %>%
    select(family, class) %>%
    left_join(df_fauna, by = c('family' = 'familia')) %>%
    tidyr::drop_na() %>%
    rename(familia = family, classe = class) %>%
    select(c(3, 4, 2, 5, 1, 6, 7)) %>%
    filter(!duplicated(especie_subespecie))

In [ ]:
head(df_fauna_fim)
dim(df_fauna_fim)

<p style="text-align:justify">A lista completa da fauna contém 1.266 espécies, porém, após a junção com os dados de classe da tabela do IRMNG observa-se que a lista passou a ter 1.210 espécies, indicando que na tabela de referência para obtenção das classes taxonômicas não existem 56 espécies da lista em questão e/ou que a lista foi gerado com nomes duplicados.</p>
    
<p style="text-align:justify">Para resolver esse problema utilizaremos o pacote <i>taxize</i>, o qual permite consultar nomes científicos em diferentes bases de dados sobre biodiversidade no mundo, a exemplo da base de do <i>Integrated Taxonomic Information System</i><a href="https://itis.gov/"> - ITS</a> e <i>The National Center for Biotechnology Information</i><a href="https://www.ncbi.nlm.nih.gov/"> - NCBI.</a> Não utilizamos este pacote desde op começo da análise em função do tempo de processamento necessário ser muito longo para a lista ora sob análise.</p>

In [ ]:
nomes_sem_classe <- df_fauna %>%
    anti_join(df_fauna_fim, by = 'familia')

In [234]:
# Obter número de linhas e colunas
dim(nomes_sem_classe)

[1] 56  6

Para ganhar desempenho no processamento da busca nos dados do ITS e NCBI, utilizaremos apenas as espécies que não possuem associação coma os dados do IRMNG.

In [ ]:
classes <- nomes_sem_classe %>%
        # Busca as classes taxonômicas na base de dados do ITS utilizando a família como parâmetro
        mutate(classe = taxize::tax_name(familia, get = "class", db = 'itis')$class) %>%
        # Busca as classes taxonômicas na base de dados do NCBI utilizando a família como parâmetro
        mutate(classe = if_else(is.na(classe), 
                                taxize::tax_name(familia, get = "class", db = 'ncbi')$class, 
                                classe)) %>%
        select(c(1, 2, 7, 3:6))

In [ ]:
dim(classes)

Após as buscas em duas bases de dados, apenas 27 espécies ficaram sem a correspondente classe taxonômica, como podemos observar na saída do código abaixo.

In [ ]:
summary(is.na(classes$classe))

De posse da maioria das classes das espécies da fauna, faremos a junção dos _dataframes_ _df_fauna_ e _classes_ para salvar os dados definitivos em uma planilha csv. Entretano, antes de salvarmos a lista definitiva, vamos ferificar quais espécies estão sem classe taxonômica e tentar obtê-las manualmente.

In [ ]:
# Junção dos dataframes
df <- rbind(classes, df_fauna_fim)

# ordenar os dados pela coluna "n"
df <- df %>%
    arrange(n)

In [230]:
df[is.na(df$classe), ]

,n,port443,classe,ordem,familia,especie_subespecie,categoria
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
506,513,*,NA,Glomeridesmida,Glomerodesmidae,Glomeridesmus spelaeus,EN
735,746,,NA,Passeriformes,Onychorhynchidae,Onychorhynchusswainsoni,VU
743,754,*,NA,Passeriformes,Platyrinchidae,Calyptura cristata,CR (PEX)
744,755,,NA,Passeriformes,Platyrinchidae,Platyrinchus leucoryphus,VU
745,756,*,NA,Passeriformes,Platyrinchidae,Platyrinchus mystaceus niveigularis,VU
752,763,,NA,Passeriformes,Rhynchocyclidae,Cnipodectessuperrufus,VU
753,764,*,NA,Passeriformes,Rhynchocyclidae,Hemitriccus kaempferi,VU
754,765,*,NA,Passeriformes,Rhynchocyclidae,Hemitriccus mirandae,EN
755,766,*,NA,Passeriformes,Rhynchocyclidae,Phylloscartes beckeri,EN


A saída do código acima mostra que as espécies sem a classe taxonômica pertencem as famílias:

In [241]:
cat(glue::glue_collapse(unique(nomes_sem_classe$familia), sep = ', ', last = ' e '))

Aromobatidae, Cycloramphidae, Hylodidae, Odontophrynidae, Glomerodesmidae, Kimulidae, Grallariidae, Onychorhynchidae, Platyrinchidae, Rhynchocyclidae, Scleruridae, Tityridae, Xenopidae, Epinephelidae, Liolaemidae, Mabuyidae, Phyllodactylidae e Sphaerodactylidae

In [249]:
df[df$familia=='Aromobatidae', 'classe'] <- 'Diplopoda'
df[df$familia=='Cycloramphidae', 'classe'] <- 'Amphibia'
df[df$familia=='Hylodidae', 'classe'] <- 'Amphibia'
df[df$familia=='Odontophrynidae', 'classe'] <- 'Amphibia'
df[df$familia=='Odontophrynidae', 'classe'] <- 'Amphibia'
df[df$familia=='Kimulidae', 'classe'] <- 'Arachnida'
df[df$familia=='Grallariidae', 'classe'] <- 'Aves'
df[df$familia=='Onychorhynchidae', 'classe'] <- 'Aves'
df[df$familia=='Platyrinchidae', 'classe'] <- 'Aves'
df[df$familia=='Scleruridae', 'classe'] <- 'Aves'
df[df$familia=='Tityridae', 'classe'] <- 'Aves'
df[df$familia=='Xenopidae', 'classe'] <- 'Aves'
df[df$familia=='Epinephelidae', 'classe'] <- 'Atheriniformes'
df[df$familia=='Liolaemidae', 'classe'] <- 'Reptilia'
df[df$familia=='Mabuyidae', 'classe'] <- 'Reptilia'
df[df$familia=='Phyllodactylidae', 'classe'] <- 'Reptilia'
df[df$familia=='Sphaerodactylidae', 'classe'] <- 'Reptilia'

In [250]:
# salvar como csv
write.csv2(df, './port_MMA_300-2022_fauna_lista_definitiva.csv',
           row.names = FALSE, 
           fileEncoding = 'latin1')